<h2>STINTSY Machine Project</h2>

Group Doc OCT (Patrick Ong, Russel Campol, Miko Tansingco)

<h4>Library Importing</h4>

In [1]:
import matplotlib.image as mpimg
import cv2
import os
import numpy as np
import pandas as pd
import collections
from tqdm import tqdm
from matplotlib import pyplot as plt
from random import *
#imports for image reading
from skimage.io import imread
from skimage.transform import resize
from sklearn.model_selection import train_test_split
#imports for label encoding for classes
from sklearn import preprocessing
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model, Model
from tensorflow.keras.optimizers import RMSprop,SGD,Adam
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization,Flatten,GlobalMaxPooling2D
from tensorflow.keras.applications.resnet_v2 import ResNet50V2, preprocess_input
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2

<h3>Encode labels

<h4>Exploratory Data Analysis</h4>

1. Analysis of images

In [2]:
data = pd.read_csv('labels.csv')

In [3]:
data

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
...,...,...
10217,ffd25009d635cfd16e793503ac5edef0,borzoi
10218,ffd3f636f7f379c51ba3648a9ff8254f,dandie_dinmont
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale
10220,ffe5f6d8e2bff356e9482a80a6e29aac,miniature_pinscher


In [4]:
data_plot = data.groupby('breed')['id'].count().sort_values()
data_plot.tail(50)

breed
weimaraner               85
bluetick                 85
african_hunting_dog      86
schipperke               86
kelpie                   86
english_foxhound         86
bouvier_des_flandres     86
collie                   87
old_english_sheepdog     87
boston_bull              87
irish_setter             88
sealyham_terrier         88
rhodesian_ridgeback      88
dandie_dinmont           89
bedlington_terrier       89
silky_terrier            90
lhasa                    90
border_terrier           91
newfoundland             91
ibizan_hound             91
pembroke                 92
italian_greyhound        92
chow                     93
pug                      94
norwegian_elkhound       95
whippet                  95
siberian_husky           95
papillon                 96
saluki                   99
lakeland_terrier         99
irish_wolfhound         101
miniature_pinscher      102
blenheim_spaniel        102
australian_terrier      102
japanese_spaniel        105
beagle        

In [5]:
data_plot.plot(kind = 'barh', figsize=(20,50), title="Count Per Breed")

TypeError: Cannot interpret '<attribute 'dtype' of 'numpy.generic' objects>' as a data type

In [6]:
data_plot.describe()

count    120.000000
mean      85.183333
std       13.298122
min       66.000000
25%       75.000000
50%       82.000000
75%       91.250000
max      126.000000
Name: id, dtype: float64

In [7]:
terrier = data[data['breed'].str.contains('terrier')]
terrier

,id,breed
5,002211c81b498ef88e1b40b9abf84e1d,bedlington_terrier
6,00290d3e1fdd27226ba27a8ce248ce85,bedlington_terrier
14,0075dc49dab4024d12fafe67074d8a81,norfolk_terrier
16,007b5a16db9d9ff9d7ad39982703e429,wire-haired_fox_terrier
18,007ff9a78eba2aebb558afea3a51c469,lakeland_terrier
...,...,...
10192,ff4afeb51a1473f7ba18669a8ff48bc9,border_terrier
10194,ff52a3909f5801a71161cec95d213107,west_highland_white_terrier
10201,ff84992beff3edd99b72718bec9448d2,bedlington_terrier
10202,ff8e3fa7e04faca99af85195507ee54d,sealyham_terrier


In [8]:
terrier['breed'].value_counts()

tibetan_terrier                   107
australian_terrier                102
lakeland_terrier                   99
border_terrier                     91
silky_terrier                      90
bedlington_terrier                 89
sealyham_terrier                   88
norfolk_terrier                    83
irish_terrier                      82
kerry_blue_terrier                 82
yorkshire_terrier                  82
scotch_terrier                     82
wire-haired_fox_terrier            82
west_highland_white_terrier        81
toy_terrier                        79
staffordshire_bullterrier          79
norwich_terrier                    78
american_staffordshire_terrier     74
soft-coated_wheaten_terrier        71
Name: breed, dtype: int64

In [9]:
terrier.shape

(1621, 2)

In [10]:
retriever = data[data['breed'].str.contains('retriever')]
retriever

,id,breed
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever
25,00a366d4b4a9bbb6c8a63126697b7656,golden_retriever
31,00cc68a50b2d016a6b29af628ea4e04b,labrador_retriever
37,0100f55e4f0fe28f2c0465d3fc4b9897,golden_retriever
127,02ff77af410e966b7b661f6f0789d947,golden_retriever
...,...,...
10134,fdc614c16f54555064a32bc94522b4a4,curly-coated_retriever
10172,febcab8eb2da444bf83336cffec7eb92,golden_retriever
10182,ff05f3976c17fef275cc0306965b3fe4,labrador_retriever
10198,ff6f47aa8e181b6efa4d0be7b09b5628,golden_retriever


In [11]:
retriever['breed'].value_counts()

labrador_retriever          84
chesapeake_bay_retriever    83
flat-coated_retriever       72
curly-coated_retriever      72
golden_retriever            67
Name: breed, dtype: int64

In [12]:
retriever.shape

(378, 2)

In [13]:
dog = data[data['breed'].str.contains('dog')]
dog

,id,breed
10,004396df1acd0f1247b740ca2b14616e,shetland_sheepdog
12,00693b8bc2470375cc744a6391d397ec,maltese_dog
15,00792e341f3c6eb33663e415d0715370,african_hunting_dog
50,014c2b0cd8e3b517e649cecf8543b8fe,african_hunting_dog
61,0177a92a906192bfde8adbb8a237e524,greater_swiss_mountain_dog
...,...,...
10154,fe3e760d763e186541e18f303cd7caca,bernese_mountain_dog
10176,fee98c990f4d69c6a8467dd0f0668440,greater_swiss_mountain_dog
10178,fef5d4cdaf50cf159102e803c7d6aa9c,bernese_mountain_dog
10200,ff7d9c08091acc3b18b869951feeb013,maltese_dog


In [14]:
dog['breed'].value_counts()

maltese_dog                   117
bernese_mountain_dog          114
old_english_sheepdog           87
african_hunting_dog            86
greater_swiss_mountain_dog     82
shetland_sheepdog              76
french_bulldog                 70
eskimo_dog                     66
Name: breed, dtype: int64

In [15]:
breeds = data['breed']
breeds = breeds.str.split('_')
breeds

0                      [boston, bull]
1                             [dingo]
2                          [pekinese]
3                          [bluetick]
4                 [golden, retriever]
                     ...             
10217                        [borzoi]
10218               [dandie, dinmont]
10219                      [airedale]
10220           [miniature, pinscher]
10221    [chesapeake, bay, retriever]
Name: breed, Length: 10222, dtype: object

In [16]:
breed_instances = [i[len(i)-1] for i in breeds]
breed_instances

['bull',
 'dingo',
 'pekinese',
 'bluetick',
 'retriever',
 'terrier',
 'terrier',
 'borzoi',
 'basenji',
 'deerhound',
 'sheepdog',
 'hound',
 'dog',
 'bluetick',
 'terrier',
 'dog',
 'terrier',
 'redbone',
 'terrier',
 'boxer',
 'doberman',
 'otterhound',
 'otterhound',
 'terrier',
 'dingo',
 'retriever',
 'schnauzer',
 'spaniel',
 'coonhound',
 'cairn',
 'affenpinscher',
 'retriever',
 'hound',
 'setter',
 'hound',
 'weimaraner',
 'schnauzer',
 'retriever',
 'bull',
 'groenendael',
 'terrier',
 'spaniel',
 'dhole',
 'dingo',
 'affenpinscher',
 'poodle',
 'terrier',
 'terrier',
 'terrier',
 'elkhound',
 'dog',
 'spaniel',
 'hound',
 'spaniel',
 'terrier',
 'schnauzer',
 'affenpinscher',
 'shih-tzu',
 'terrier',
 'kuvasz',
 'shepherd',
 'dog',
 'basset',
 'terrier',
 'schipperke',
 'ridgeback',
 'setter',
 'appenzeller',
 'shepherd',
 'bloodhound',
 'samoyed',
 'schnauzer',
 'spaniel',
 'setter',
 'kelpie',
 'papillon',
 'collie',
 'spaniel',
 'appenzeller',
 'deerhound',
 'entlebuche

In [17]:
breed_instances_series = pd.Series(breed_instances)

In [18]:
breed_instances_series.value_counts().head(30)

terrier        1542
spaniel         589
dog             465
retriever       378
hound           276
setter          252
poodle          238
schnauzer       219
sheepdog        163
collie          159
mastiff         144
deerhound       126
entlebucher     115
shih-tzu        112
pyrenees        111
pomeranian      111
basenji         110
samoyed         109
airedale        107
leonberg        106
cairn           106
beagle          105
pinscher        102
wolfhound       101
saluki           99
papillon         96
husky            95
elkhound         95
whippet          95
pug              94
dtype: int64

In [19]:
breed_instances_series.value_counts().tail(30)

keeshond         81
malamute         81
hairless         80
clumber          80
affenpinscher    80
dingo            80
bullterrier      79
appenzeller      78
coonhound        77
dhole            76
cardigan         76
rottweiler       76
boxer            75
dane             75
borzoi           75
pekinese         75
springer         75
pointer          75
doberman         74
malinois         73
redbone          72
chihuahua        71
kuvasz           71
vizsla           70
bulldog          70
shepherd         69
otterhound       69
komondor         67
griffon          67
briard           66
dtype: int64

In [20]:
breeds_unique = data['breed'].unique()
breeds_unique

for i in range(len(breeds_unique)):
    j = breeds_unique[i].split('_')
    breeds_unique[i] = j[len(j)-1]
    
pd.Series(breeds_unique).value_counts().head(30)

terrier         18
spaniel          7
retriever        5
dog              5
poodle           3
schnauzer        3
hound            3
setter           3
collie           2
sheepdog         2
mastiff          2
newfoundland     1
dinmont          1
bull             1
kuvasz           1
schipperke       1
bluetick         1
pomeranian       1
husky            1
coonhound        1
pointer          1
elkhound         1
beagle           1
borzoi           1
lhasa            1
dingo            1
malamute         1
weimaraner       1
malinois         1
dane             1
dtype: int64

In [21]:
data['breed'].unique()

array(['boston_bull', 'dingo', 'pekinese', 'bluetick', 'golden_retriever',
       'bedlington_terrier', 'borzoi', 'basenji', 'scottish_deerhound',
       'shetland_sheepdog', 'walker_hound', 'maltese_dog',
       'norfolk_terrier', 'african_hunting_dog',
       'wire-haired_fox_terrier', 'redbone', 'lakeland_terrier', 'boxer',
       'doberman', 'otterhound', 'standard_schnauzer',
       'irish_water_spaniel', 'black-and-tan_coonhound', 'cairn',
       'affenpinscher', 'labrador_retriever', 'ibizan_hound',
       'english_setter', 'weimaraner', 'giant_schnauzer', 'groenendael',
       'dhole', 'toy_poodle', 'border_terrier', 'tibetan_terrier',
       'norwegian_elkhound', 'shih-tzu', 'irish_terrier', 'kuvasz',
       'german_shepherd', 'greater_swiss_mountain_dog', 'basset',
       'australian_terrier', 'schipperke', 'rhodesian_ridgeback',
       'irish_setter', 'appenzeller', 'bloodhound', 'samoyed',
       'miniature_schnauzer', 'brittany_spaniel', 'kelpie', 'papillon',
       'borde

<h2> Data Preprocessing </h2>

In [22]:
toRemove = pd.DataFrame()
for i in data['breed'].unique():
    length = len(data[data['breed'] == i])
    if length < 90:
        toRemove = toRemove.append(data[data['breed'] == i])
toRemove

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
38,010d65bd29d246aea53d9849da142ccf,boston_bull
304,0755a940eac9a9b8cf0328b4be062096,boston_bull
324,07d7cfbdbd3682cae50902b53c798028,boston_bull
471,0b3d3120a62c2bc280f589d473041d06,boston_bull
...,...,...
9632,f25a4184e33741e745175fd1d7c6d172,rottweiler
9823,f72df3c3daa677aa76027366ad55721f,rottweiler
9870,f8271e8e638b9bab6a0d3e164874dd53,rottweiler
9997,fab782d25875a7cf5298cd2e2aa01cd5,rottweiler


In [23]:
new_data = data.drop(labels=toRemove.index, axis=0)
new_data

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
12,00693b8bc2470375cc744a6391d397ec,maltese_dog
18,007ff9a78eba2aebb558afea3a51c469,lakeland_terrier
29,00bee065dcec471f26394855c5c2f3de,cairn
...,...,...
10212,ffc532991d3cd7880d27a449ed1c4770,tibetan_terrier
10215,ffcde16e7da0872c357fbc7e2168c05f,airedale
10216,ffcffab7e4beef9a9b8076ef2ca51909,samoyed
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale


<h3>check new dataset

In [24]:
new_data

,id,breed
8,003df8b8a8b05244b1d920bb6cf451f9,basenji
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound
12,00693b8bc2470375cc744a6391d397ec,maltese_dog
18,007ff9a78eba2aebb558afea3a51c469,lakeland_terrier
29,00bee065dcec471f26394855c5c2f3de,cairn
...,...,...
10212,ffc532991d3cd7880d27a449ed1c4770,tibetan_terrier
10215,ffcde16e7da0872c357fbc7e2168c05f,airedale
10216,ffcffab7e4beef9a9b8076ef2ca51909,samoyed
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale


In [25]:
data_plot = new_data.groupby('breed')['id'].count().sort_values()

In [26]:
data_plot

breed
silky_terrier            90
lhasa                    90
ibizan_hound             91
newfoundland             91
border_terrier           91
pembroke                 92
italian_greyhound        92
chow                     93
pug                      94
norwegian_elkhound       95
whippet                  95
siberian_husky           95
papillon                 96
saluki                   99
lakeland_terrier         99
irish_wolfhound         101
australian_terrier      102
blenheim_spaniel        102
miniature_pinscher      102
beagle                  105
japanese_spaniel        105
leonberg                106
cairn                   106
tibetan_terrier         107
airedale                107
samoyed                 109
basenji                 110
great_pyrenees          111
pomeranian              111
shih-tzu                112
bernese_mountain_dog    114
entlebucher             115
afghan_hound            116
maltese_dog             117
scottish_deerhound      126
Name: id, dtyp

<h3>Initialize variables

In [27]:
num_breeds = 35
im_size = 299
batch_size = 64
encoder = preprocessing.LabelEncoder()

In [28]:
new_data['img_file'] = new_data['id'].apply(lambda x: x + ".jpg")

In [29]:
new_data

,id,breed,img_file
8,003df8b8a8b05244b1d920bb6cf451f9,basenji,003df8b8a8b05244b1d920bb6cf451f9.jpg
9,0042188c895a2f14ef64a918ed9c7b64,scottish_deerhound,0042188c895a2f14ef64a918ed9c7b64.jpg
12,00693b8bc2470375cc744a6391d397ec,maltese_dog,00693b8bc2470375cc744a6391d397ec.jpg
18,007ff9a78eba2aebb558afea3a51c469,lakeland_terrier,007ff9a78eba2aebb558afea3a51c469.jpg
29,00bee065dcec471f26394855c5c2f3de,cairn,00bee065dcec471f26394855c5c2f3de.jpg
...,...,...,...
10212,ffc532991d3cd7880d27a449ed1c4770,tibetan_terrier,ffc532991d3cd7880d27a449ed1c4770.jpg
10215,ffcde16e7da0872c357fbc7e2168c05f,airedale,ffcde16e7da0872c357fbc7e2168c05f.jpg
10216,ffcffab7e4beef9a9b8076ef2ca51909,samoyed,ffcffab7e4beef9a9b8076ef2ca51909.jpg
10219,ffe2ca6c940cddfee68fa3cc6c63213f,airedale,ffe2ca6c940cddfee68fa3cc6c63213f.jpg


<h3> Open images and pre processing

In [30]:
#create a numpy array of the shape
#(number of dataset records, image size , image size, 3 for rgb channel ayer)
#this will be input for model
train_x = np.zeros((len(new_data), im_size, im_size, 3), dtype='float32')
train_file = 'train/' 
#iterate over img_file column of our dataset
for i, img_id in enumerate(new_data['img_file']):
  #read the image file and convert into numeric format
  #resize all images to one dimension i.e. 224x224
  #we will get array with the shape of
  # (224,224,3) where 3 is the RGB channels layers
    img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  #scale array into the range of -1 to 1.
  #preprocess the array and expand its dimension on the axis 0 
    img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  #update the train_x variable with new element
    train_x[i] = img_array

In [31]:
#This will be the target for the model.
#convert breed names into numerical format
train_y = encoder.fit_transform(new_data["breed"].values)

In [32]:
train_y

array([ 3, 29, 19, ..., 28,  1, 20])

<h3>Splitting into train and test

In [33]:
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42,stratify=train_y)
print("training data: (",x_train.shape, ",",y_train.shape,")")
print("test data: (",x_test.shape, ",",y_test.shape,")")

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

training data: ( (2869, 299, 299, 3) , (2869,) )
test data: ( (718, 299, 299, 3) , (718,) )


<h3>Splitting into train and validation

In [34]:
x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.4,random_state=42,stratify=y_train)
print("training data: (",x_train.shape, ",",y_train.shape,")")
print("test data: (",x_val.shape, ",",y_val.shape,")")

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

training data: ( (1721, 299, 299, 3) , (1721,) )
test data: ( (1148, 299, 299, 3) , (1148,) )


<h3>Data augmentation

In [35]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
 
#generate images for training sets 
train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=batch_size)
 
#same process for Testing sets also by declaring the instance
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_val, 
                                     y_val, 
                                     batch_size=batch_size)

<h3>building the model

In [36]:
#building the model using ResNet50V2 with input shape of our image array
#weights for our network will be from of imagenet dataset
#we will not include the first Dense layer
resnet = ResNet50V2(input_shape = [im_size,im_size,3], weights='imagenet', include_top=False)
#freeze all trainable layers and train only top layers 
for layer in resnet.layers:
    layer.trainable = False
 
    #add global average pooling layer and Batch Normalization layer
x = resnet.output
x = BatchNormalization()(x)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
#add fully connected layer
x = Dense(1024, activation='relu')(x)
x = Dropout(0.5)(x)

In [37]:
#add output layer having the shape equal to number of breeds
predictions = Dense(num_breeds, activation='softmax')(x)
 
#create model class with inputs and outputs
model = Model(inputs=resnet.input, outputs=predictions)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 305, 305, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 150, 150, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 75, 75, 256)  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 75, 75, 64)   16384       ['conv2_block3_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv2_block3_1_bn (BatchNormal  (None, 75, 75, 64)  256         ['conv2_block3_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_blo

 conv3_block2_2_conv (Conv2D)   (None, 38, 38, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 38, 38, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block2_2_relu (Activatio  (None, 38, 38, 128)  0          ['conv3_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_3_conv (Conv2D)   (None, 38, 38, 512)  66048       ['conv3_block2_2_relu[0][0]']    
                                                                                                  
 conv3_blo

 conv4_block1_1_conv (Conv2D)   (None, 19, 19, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block1_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block1_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block1_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block1_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block1_2_pad (ZeroPaddin  (None, 21, 21, 256)  0          ['conv4_block1_1_relu[0][0]']    
 g2D)     

                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block3_3_conv (Conv2D)   (None, 19, 19, 1024  263168      ['conv4_block3_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv4_block3_out (Add)         (None, 19, 19, 1024  0           ['conv4_block2_out[0][0]',       
                                )                                 'conv4_block3_3_conv[0][0]']    
                                                                                                  
 conv4_blo

 conv4_block6_1_bn (BatchNormal  (None, 19, 19, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block6_1_relu (Activatio  (None, 19, 19, 256)  0          ['conv4_block6_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block6_2_pad (ZeroPaddin  (None, 21, 21, 256)  0          ['conv4_block6_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv4_block6_2_conv (Conv2D)   (None, 10, 10, 256)  589824      ['conv4_block6_2_pad[0][0]']     
          

                                                                                                  
 conv5_block2_2_relu (Activatio  (None, 10, 10, 512)  0          ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv5_block2_3_conv (Conv2D)   (None, 10, 10, 2048  1050624     ['conv5_block2_2_relu[0][0]']    
                                )                                                                 
                                                                                                  
 conv5_block2_out (Add)         (None, 10, 10, 2048  0           ['conv5_block1_out[0][0]',       
                                )                                 'conv5_block2_3_conv[0][0]']    
                                                                                                  
 conv5_blo

<h3> training

In [38]:
#epochs for model training and learning rate for optimizer
epochs = 20
learning_rate = 1e-3
 
#using RMSprop optimizer to compile or build the model
optimizer = RMSprop(learning_rate=learning_rate,rho=0.9)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])
 
#fit the training generator data and train the model
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_val.shape[0] // batch_size)
 
#Save the model for prediction
model.save("model")

Epoch 1/20
26/26 [==============================] - 261s 10s/step - loss: 1.9723 - accuracy: 0.5196 - val_loss: 0.5342 - val_accuracy: 0.8906
Epoch 2/20
26/26 [==============================] - 254s 10s/step - loss: 0.8183 - accuracy: 0.7809 - val_loss: 0.3589 - val_accuracy: 0.8961
Epoch 3/20
26/26 [==============================] - 249s 10s/step - loss: 0.6094 - accuracy: 0.8117 - val_loss: 0.3224 - val_accuracy: 0.9017
Epoch 4/20
26/26 [==============================] - 250s 10s/step - loss: 0.4614 - accuracy: 0.8588 - val_loss: 0.3104 - val_accuracy: 0.8980
Epoch 5/20
26/26 [==============================] - 249s 10s/step - loss: 0.4362 - accuracy: 0.8612 - val_loss: 0.2785 - val_accuracy: 0.9099
Epoch 6/20
26/26 [==============================] - 253s 10s/step - loss: 0.3778 - accuracy: 0.8763 - val_loss: 0.2764 - val_accuracy: 0.9118
Epoch 7/20
26/26 [==============================] - 254s 10s/step - loss: 0.3192 - accuracy: 0.8998 - val_loss: 0.2701 - val_accuracy: 0.9090
Epoch 

INFO:tensorflow:Assets written to: model\assets


INFO:tensorflow:Assets written to: model\assets


<h4> predictions on test data </h4>

In [39]:
predictions=model.predict(x_test)
predictions

23/23 [==============================] - 75s 3s/step


array([[2.5694960e-10, 1.8283743e-10, 3.8160708e-10, ..., 9.8643825e-13,
        4.0446344e-11, 1.5148515e-09],
       [1.8192203e-07, 5.4273332e-08, 1.4120588e-03, ..., 7.6872384e-05,
        4.8420406e-03, 1.6745224e-07],
       [5.1661851e-07, 2.4053001e-11, 8.0788859e-13, ..., 8.2782360e-13,
        3.8927715e-12, 3.2796692e-08],
       ...,
       [4.4498988e-06, 2.3042020e-08, 4.4576223e-09, ..., 5.7272903e-10,
        1.6633894e-06, 3.2631178e-07],
       [3.7298416e-04, 1.0384539e-07, 1.6819366e-09, ..., 1.7639131e-08,
        1.0481782e-08, 1.8992481e-01],
       [1.3309469e-08, 1.3216409e-07, 6.3732495e-09, ..., 5.2828819e-10,
        7.5024261e-08, 7.1125654e-11]], dtype=float32)

In [40]:
probs = np.argmax(predictions, axis=1)
probs

array([28, 19, 17, 19, 24,  3, 21, 19, 19, 28, 22, 14,  9, 16, 27, 22, 29,
       11, 12,  7,  7,  0,  8, 20,  7, 15,  7, 32, 18, 20, 15,  6, 30, 32,
       25, 22, 12, 15, 30,  0, 17, 19, 34, 34, 11, 19, 10,  1, 19, 22, 11,
       26, 13, 22, 24,  1, 15,  3, 27,  5,  7,  0,  8, 15, 10, 12,  4,  2,
       34, 23, 27, 26, 33,  4,  7, 11, 31, 14,  1, 20,  4, 31, 31, 25, 33,
       16, 13, 20,  7,  9, 26, 25,  1, 10,  4,  9,  6, 16, 11, 34, 26, 17,
        6,  8,  6, 28, 29, 12,  2,  1, 10, 21,  4,  9, 31,  0, 11, 14, 20,
       34,  7, 20, 30, 28,  9, 26,  2, 34, 18, 21,  4,  7, 12,  1,  1, 21,
        2, 21,  3, 19, 18, 23,  7,  8, 18, 30, 13, 20, 23, 23, 32, 12, 25,
       21, 15, 22, 30, 33,  0, 34, 10,  3, 10, 28, 28, 32, 27, 25, 11,  5,
        6,  0, 30, 18, 30, 20, 18, 12, 14,  6, 24, 11, 19, 25, 10, 29, 19,
       17, 17, 33,  3, 10, 29,  6, 28,  0, 21, 15,  9, 21,  8,  9, 20, 19,
       11, 10, 22, 22, 23, 14,  1, 23, 31, 25, 17, 23, 28, 25,  0, 29, 32,
       26, 11, 14, 10,  3

In [41]:
y_test

array([28, 19, 17, 19, 24,  3, 21, 19, 19, 28, 22, 14,  9, 16, 27, 22, 29,
       11, 12, 18,  7,  0,  8, 20,  7, 15,  7,  2, 18, 20, 15,  6, 30, 32,
       25, 22, 12, 15, 30,  0, 17, 19, 34, 34, 11, 19, 10,  1, 19, 22, 11,
       26, 13, 22, 24,  1, 15,  3, 27,  5,  8,  0,  8, 15, 10, 12,  4, 32,
       34, 23, 27, 26, 33,  4, 16, 11, 12, 14,  1, 20,  4, 31, 31, 25, 33,
       16, 29, 20,  7,  9, 26, 25,  1, 10,  4,  9,  6, 16, 11, 34, 26, 17,
        6,  8,  6, 28, 29, 12,  2,  1, 10, 21,  4,  9, 31, 29, 11,  3, 20,
       14,  7, 20, 30, 28,  9, 26, 32, 34, 18, 17,  4,  7, 12,  1,  1, 21,
        2, 21,  3, 19, 18, 23,  7,  8, 18, 30, 13, 20, 25, 23, 32, 12, 25,
       21, 15, 22, 30, 33,  0, 34, 10,  3, 10, 28, 28,  8, 27, 25, 11,  5,
        6,  0, 30, 18, 15, 20, 18, 12, 14,  6, 24, 28, 19, 25, 10, 13, 19,
       17, 17, 33,  3, 10, 13,  6, 28,  0, 21, 15, 22, 21,  8,  9, 20, 19,
       11, 10, 22, 22, 23, 14,  1, 23, 31, 28, 17, 23, 28, 25, 13, 29, 32,
       31, 11, 14, 10,  3

In [42]:
num_correct = (probs == y_test).sum()
print(num_correct.item())

646


In [43]:
accuracy = num_correct/len(y_test)
print('{:.4f}'.format(accuracy.item()))

0.8997


<h2>Image loading and preprocessing for  InceptionResNetV2 and InceptionV3

<h4>since the size of images in resnet is different from what is required in the next 2 models we need to do data pre-processing again.

In [44]:
im_size = 299 #image size needed for both inceptionRESNetV2 and inceptionV3

In [45]:
#create a numpy array of the shape
#(number of dataset records, image size , image size, 3 for rgb channel ayer)
#this will be input for model
train_x = np.zeros((len(new_data), im_size, im_size, 3), dtype='float32')
train_file = 'train/' 
#iterate over img_file column of our dataset
for i, img_id in enumerate(new_data['img_file']):
  #read the image file and convert into numeric format
  #resize all images to one dimension i.e. 224x224
  #we will get array with the shape of
  # (224,224,3) where 3 is the RGB channels layers
    img = cv2.resize(cv2.imread(train_file+img_id,cv2.IMREAD_COLOR),((im_size,im_size)))
  #scale array into the range of -1 to 1.
  #preprocess the array and expand its dimension on the axis 0 
    img_array = preprocess_input(np.expand_dims(np.array(img[...,::-1].astype(np.float32)).copy(), axis=0))
  #update the train_x variable with new element
    train_x[i] = img_array

In [46]:
train_y = encoder.fit_transform(new_data["breed"].values)

In [47]:
x_train, x_test, y_train, y_test = train_test_split(train_x,train_y,test_size=0.2,random_state=42,stratify=train_y)
print("training data: (",x_train.shape, ",",y_train.shape,")")
print("test data: (",x_test.shape, ",",y_test.shape,")")

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

training data: ( (2869, 299, 299, 3) , (2869,) )
test data: ( (718, 299, 299, 3) , (718,) )


<h3>Split training into train and validation

In [48]:

x_train, x_val, y_train, y_val = train_test_split(x_train,y_train,test_size=0.4,random_state=42,stratify=y_train)
print("training data: (",x_train.shape, ",",y_train.shape,")")
print("test data: (",x_val.shape, ",",y_val.shape,")")

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  return floored.astype(np.int)
C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\sklearn\utils\__init__.py:806: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current u

training data: ( (1721, 299, 299, 3) , (1721,) )
test data: ( (1148, 299, 299, 3) , (1148,) )


In [49]:
train_datagen = ImageDataGenerator(rotation_range=45,
                                   width_shift_range=0.2,
                                   height_shift_range=0.2,
                                   shear_range=0.2,
                                   zoom_range=0.25,
                                   horizontal_flip=True,
                                   fill_mode='nearest')
 
#generate images for training sets 
train_generator = train_datagen.flow(x_train, 
                                     y_train, 
                                     batch_size=batch_size)
 
#same process for Testing sets also by declaring the instance
test_datagen = ImageDataGenerator()
 
test_generator = test_datagen.flow(x_val, 
                                     y_val, 
                                     batch_size=batch_size)

<h2>InceptionResNet V2 model

In [50]:
IRnet = InceptionResNetV2(weights='imagenet', include_top=False, input_shape = [im_size,im_size,3])
IRnet.summary()

219055592/219055592 [==============================] - 34s 0us/step
Model: "inception_resnet_v2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 rmalization

 conv2d_11 (Conv2D)             (None, 35, 35, 64)   12288       ['average_pooling2d[0][0]']      
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 35, 35, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_11 (BatchN  (None, 35, 35, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
          

 ormalization)                                                                                    
                                                                                                  
 activation_21 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 35, 35, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 35, 35, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 35, 35, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
          

 batch_normalization_30 (BatchN  (None, 35, 35, 64)  192         ['conv2d_29[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_24 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_25[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_27[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 35, 35, 64)   0           ['batch_normalization_30[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 35, 35, 128)  0           ['activation_24[0][0]',          
          

                                                                                                  
 batch_normalization_38 (BatchN  (None, 35, 35, 32)  96          ['conv2d_37[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_41 (BatchN  (None, 35, 35, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_38[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 35, 35, 48)   0           ['batch_normalization_41[0][0]'] 
          

...

 batch_normalization_170 (Batch  (None, 8, 8, 224)   672         ['conv2d_169[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_169 (Activation)    (None, 8, 8, 224)    0           ['batch_normalization_170[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 8, 8, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 8, 8, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_168 (Batch  (None, 8, 8, 192)   576         ['conv2d_167[0][0]']             
 Normaliza

 conv2d_178 (Conv2D)            (None, 8, 8, 256)    172032      ['activation_177[0][0]']         
                                                                                                  
 batch_normalization_176 (Batch  (None, 8, 8, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_179 (Batch  (None, 8, 8, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 8, 8, 192)    0           ['batch_normalization_176[0][0]']
                                                                                                  
 activatio

                                                                                                  
 activation_183 (Activation)    (None, 8, 8, 192)    0           ['batch_normalization_184[0][0]']
                                                                                                  
 activation_186 (Activation)    (None, 8, 8, 256)    0           ['batch_normalization_187[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 8, 8, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 8, 8, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 

                                                                                                  
 block8_8_conv (Conv2D)         (None, 8, 8, 2080)   933920      ['block8_8_mixed[0][0]']         
                                                                                                  
 block8_8 (Lambda)              (None, 8, 8, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 8, 8, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_196 (Conv2D)            (None, 8, 8, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_nor

                                                                                                  
 conv_7b_bn (BatchNormalization  (None, 8, 8, 1536)  4608        ['conv_7b[0][0]']                
 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 8, 8, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
Total params: 54,336,736
Trainable params: 54,276,192
Non-trainable params: 60,544
__________________________________________________________________________________________________


In [51]:
for layer in IRnet.layers:
    layer.trainable = False

x = IRnet.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_breeds, activation='softmax')(x)

model= Model(inputs=IRnet.input, outputs=predictions)
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_2[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_1 (BatchNo  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 rmalization)                   )                                                           

 batch_normalization_6 (BatchNo  (None, 35, 35, 96)  288         ['conv2d_5[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_8 (BatchNo  (None, 35, 35, 64)  192         ['conv2d_7[0][0]']               
 rmalization)                                                                                     
                                                                                                  
 batch_normalization_11 (BatchN  (None, 35, 35, 96)  288         ['conv2d_10[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_12 (BatchN  (None, 35, 35, 64)  192         ['conv2d_11[0][0]']              
 ormalizat

 activation_21 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_22[0][0]'] 
                                                                                                  
 conv2d_19 (Conv2D)             (None, 35, 35, 32)   10240       ['block35_1_ac[0][0]']           
                                                                                                  
 conv2d_22 (Conv2D)             (None, 35, 35, 48)   13824       ['activation_21[0][0]']          
                                                                                                  
 batch_normalization_20 (BatchN  (None, 35, 35, 32)  96          ['conv2d_19[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 batch_normalization_23 (BatchN  (None, 35, 35, 48)  144         ['conv2d_22[0][0]']              
 ormalizat

                                                                                                  
 activation_24 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_25[0][0]'] 
                                                                                                  
 activation_26 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_27[0][0]'] 
                                                                                                  
 activation_29 (Activation)     (None, 35, 35, 64)   0           ['batch_normalization_30[0][0]'] 
                                                                                                  
 block35_3_mixed (Concatenate)  (None, 35, 35, 128)  0           ['activation_24[0][0]',          
                                                                  'activation_26[0][0]',          
                                                                  'activation_29[0][0]']          
          

 ormalization)                                                                                    
                                                                                                  
 batch_normalization_41 (BatchN  (None, 35, 35, 48)  144         ['conv2d_40[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_37 (Activation)     (None, 35, 35, 32)   0           ['batch_normalization_38[0][0]'] 
                                                                                                  
 activation_40 (Activation)     (None, 35, 35, 48)   0           ['batch_normalization_41[0][0]'] 
                                                                                                  
 conv2d_36 (Conv2D)             (None, 35, 35, 32)   10240       ['block35_4_ac[0][0]']           
          

...

                                                                                                  
 activation_169 (Activation)    (None, 8, 8, 224)    0           ['batch_normalization_170[0][0]']
                                                                                                  
 conv2d_167 (Conv2D)            (None, 8, 8, 192)    399360      ['block8_1_ac[0][0]']            
                                                                                                  
 conv2d_170 (Conv2D)            (None, 8, 8, 256)    172032      ['activation_169[0][0]']         
                                                                                                  
 batch_normalization_168 (Batch  (None, 8, 8, 192)   576         ['conv2d_167[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_nor

 batch_normalization_176 (Batch  (None, 8, 8, 192)   576         ['conv2d_175[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 batch_normalization_179 (Batch  (None, 8, 8, 256)   768         ['conv2d_178[0][0]']             
 Normalization)                                                                                   
                                                                                                  
 activation_175 (Activation)    (None, 8, 8, 192)    0           ['batch_normalization_176[0][0]']
                                                                                                  
 activation_178 (Activation)    (None, 8, 8, 256)    0           ['batch_normalization_179[0][0]']
                                                                                                  
 block8_4_

                                                                                                  
 activation_186 (Activation)    (None, 8, 8, 256)    0           ['batch_normalization_187[0][0]']
                                                                                                  
 block8_6_mixed (Concatenate)   (None, 8, 8, 448)    0           ['activation_183[0][0]',         
                                                                  'activation_186[0][0]']         
                                                                                                  
 block8_6_conv (Conv2D)         (None, 8, 8, 2080)   933920      ['block8_6_mixed[0][0]']         
                                                                                                  
 block8_6 (Lambda)              (None, 8, 8, 2080)   0           ['block8_5_ac[0][0]',            
                                                                  'block8_6_conv[0][0]']          
          

                                                                                                  
 block8_8 (Lambda)              (None, 8, 8, 2080)   0           ['block8_7_ac[0][0]',            
                                                                  'block8_8_conv[0][0]']          
                                                                                                  
 block8_8_ac (Activation)       (None, 8, 8, 2080)   0           ['block8_8[0][0]']               
                                                                                                  
 conv2d_196 (Conv2D)            (None, 8, 8, 192)    399360      ['block8_8_ac[0][0]']            
                                                                                                  
 batch_normalization_197 (Batch  (None, 8, 8, 192)   576         ['conv2d_196[0][0]']             
 Normalization)                                                                                   
          

 )                                                                                                
                                                                                                  
 conv_7b_ac (Activation)        (None, 8, 8, 1536)   0           ['conv_7b_bn[0][0]']             
                                                                                                  
 global_average_pooling2d_1 (Gl  (None, 1536)        0           ['conv_7b_ac[0][0]']             
 obalAveragePooling2D)                                                                            
                                                                                                  
 dense_2 (Dense)                (None, 35)           53795       ['global_average_pooling2d_1[0][0
                                                                 ]']                              
                                                                                                  
Total para

In [54]:
epochs = 10

 
#using SGD to compile or build the model
optimizer = SGD(lr=0.1, momentum=0.95, nesterov=False)
model.compile(optimizer=optimizer,
              loss='sparse_categorical_crossentropy',
              metrics=["accuracy"])

In [55]:

 
#fit the training generator data and train the model
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_val.shape[0] // batch_size)

Epoch 1/10
26/26 [==============================] - 434s 17s/step - loss: 1.1102 - accuracy: 0.7381 - val_loss: 0.1711 - val_accuracy: 0.9494
Epoch 2/10
26/26 [==============================] - 419s 16s/step - loss: 0.4308 - accuracy: 0.9089 - val_loss: 0.3012 - val_accuracy: 0.9421
Epoch 3/10
26/26 [==============================] - 418s 16s/step - loss: 0.5608 - accuracy: 0.9034 - val_loss: 0.2758 - val_accuracy: 0.9458
Epoch 4/10
26/26 [==============================] - 417s 16s/step - loss: 0.5121 - accuracy: 0.9053 - val_loss: 0.3585 - val_accuracy: 0.9412
Epoch 5/10
26/26 [==============================] - 422s 17s/step - loss: 0.4437 - accuracy: 0.9203 - val_loss: 0.2693 - val_accuracy: 0.9439
Epoch 6/10
26/26 [==============================] - 419s 16s/step - loss: 0.3940 - accuracy: 0.9149 - val_loss: 0.3438 - val_accuracy: 0.9375
Epoch 7/10
26/26 [==============================] - 419s 16s/step - loss: 0.3769 - accuracy: 0.9264 - val_loss: 0.3319 - val_accuracy: 0.9430
Epoch 

In [56]:
predictions=model.predict(x_test)
probs = np.argmax(predictions, axis=1)
num_correct = (probs == y_test).sum()
accuracy = num_correct/len(y_test)
print('{:.4f}'.format(accuracy.item()))

23/23 [==============================] - 128s 5s/step
0.9443


<h2>InceptionV3 model

In [57]:
base_model = InceptionV3(weights = 'imagenet', include_top = False, input_shape=(299, 299, 3))

In [58]:
for layer in base_model.layers:
    layer.trainable = False
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_breeds, activation='softmax')(x)
model = Model(inputs = base_model.input, outputs = predictions)

In [59]:
model.compile(Adam(lr=.0001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

C:\Users\Miko Tansingco\Anaconda3\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:110: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


In [60]:
epochs=10
hist = model.fit(train_generator,
                 steps_per_epoch= x_train.shape[0] // batch_size,
                 epochs= epochs,
                 validation_data= test_generator,
                 validation_steps= x_val.shape[0] // batch_size)

Epoch 1/10
26/26 [==============================] - 212s 8s/step - loss: 3.0474 - accuracy: 0.3627 - val_loss: 2.3026 - val_accuracy: 0.8070
Epoch 2/10
26/26 [==============================] - 180s 7s/step - loss: 1.8610 - accuracy: 0.8027 - val_loss: 1.1962 - val_accuracy: 0.8980
Epoch 3/10
26/26 [==============================] - 178s 7s/step - loss: 1.0073 - accuracy: 0.8618 - val_loss: 0.6003 - val_accuracy: 0.9283
Epoch 4/10
26/26 [==============================] - 178s 7s/step - loss: 0.6474 - accuracy: 0.8805 - val_loss: 0.3781 - val_accuracy: 0.9467
Epoch 5/10
26/26 [==============================] - 179s 7s/step - loss: 0.4853 - accuracy: 0.9016 - val_loss: 0.3012 - val_accuracy: 0.9375
Epoch 6/10
26/26 [==============================] - 178s 7s/step - loss: 0.4184 - accuracy: 0.9053 - val_loss: 0.2497 - val_accuracy: 0.9568
Epoch 7/10
26/26 [==============================] - 177s 7s/step - loss: 0.3597 - accuracy: 0.9131 - val_loss: 0.2239 - val_accuracy: 0.9540
Epoch 8/10
26

In [61]:
predictions=model.predict(x_test)
probs = np.argmax(predictions, axis=1)
num_correct = (probs == y_test).sum()
accuracy = num_correct/len(y_test)
print('{:.4f}'.format(accuracy.item()))

23/23 [==============================] - 49s 2s/step
0.9596
